### Task
Write a function that takes two Pandas dataframes as inputs and returns a single dataframe. The task order is:
1. For each store_id, calculate the individual revenue percentage, called revenue_shares, of every product that makes up the store_id's total revenue.
2. Sort the results by the store_id in ascending order and by the products revenue_shares in descending order.
3. Reset the index of output dataframe to start with 0.

In [1]:
import pandas as pd

In [2]:
# First create dummy dataset
# Sales
store_id = ['1','1','2','2','2']
product_id = ['M1','M3','M2','M3','M4']
units = [10,50,10,20,15]
d = {
    'store_id': pd.Series(store_id),
    'product_id': pd.Series(product_id),
    'units': pd.Series(units),
    }
sales = pd.DataFrame(d)

# Price
product_id = ['M1','M2','M3','M4','M5']
gbp_price = [100,500,300,50,100]
d = {
    'product_id': pd.Series(product_id),
    'gbp_price': pd.Series(gbp_price),
    }
price = pd.DataFrame(d)

In [3]:
print(sales), print(price)

  store_id product_id  units
0        1         M1     10
1        1         M3     50
2        2         M2     10
3        2         M3     20
4        2         M4     15
  product_id  gbp_price
0         M1        100
1         M2        500
2         M3        300
3         M4         50
4         M5        100


(None, None)

In [4]:
def calc_revenue_share(df_sales, df_price, rounding_pct=2):
    
    #-- First copy the sales dataframe, try deep copy
    df_revenue_share = df_sales.copy()
    
    #-- Look up and create new indices for gbp_price and revenue_share
    df_revenue_share['gbp_price'] = df_revenue_share['product_id'].map(df_price.set_index('product_id')['gbp_price'],na_action='ignore')
    df_revenue_share['revenue'] = df_revenue_share.gbp_price * df_revenue_share.units
    
    """
      Compute percentages of revenue. For convenience, we will use 'transform'
      since it returns a list with the same length as the original dataframe entry.
    
      Meaning of "revenue_share.groupby('store_id')['revenue'].transform('sum')":
        Group the revenue_share dataframe by store_id and use the key ['revenue'].
        Then apply the transformation ('sum').
    
      Ref:
        1. https://stackoverflow.com/a/57359372
        2. https://pbpython.com/pandas_transform.html
    """
    df_revenue_share['revenue_share'] = 100 * df_revenue_share['revenue'] / df_revenue_share.groupby('store_id')['revenue'].transform('sum')

    return df_revenue_share \
            .round({'revenue_share':rounding_pct}) \
            .sort_values(by=['store_id','revenue_share'],ascending=[True,False]) \
            .reset_index(drop=True)

In [5]:
calc_revenue_share(sales,price)

,store_id,product_id,units,gbp_price,revenue,revenue_share
0,1,M3,50,300,15000,93.75
1,1,M1,10,100,1000,6.25
2,2,M3,20,300,6000,51.06
3,2,M2,10,500,5000,42.55
4,2,M4,15,50,750,6.38
